In [2]:
import pandas as pd
import numpy as np

In [20]:
sept = pd.read_pickle('pickles/sept.pickle')
sept

,text,str,verse,chapter,book,rmac
0,εν,1722,1,1,1,prep
1,αρχη,746,1,1,1,n-dsf
2,εποιησεν,4160,1,1,1,v-aai-3s
3,ο,3588,1,1,1,t-nsm
4,θεος,2316,1,1,1,n-nsm
...,...,...,...,...,...,...
475394,ελθω,2064,6,4,39,v-aas-1s
475395,και,2532,6,4,39,conj
475396,παταξω,3960,6,4,39,v-fai-1s
475397,την,3588,6,4,39,t-asf
